In [1]:
import pandas as pd
import csv

In [2]:
url = 'https://storage.gra.cloud.ovh.net/v1/AUTH_32c5d10cb0fe4519b957064a111717e3/bso_dump/bso-clinical-trials_20240711.jsonl.gz'
df = pd.read_json(url, lines=True)

In [3]:
df = df[df.all_sources.apply(lambda x: 'clinical_trials' in x or 'euctr' in x)]
len(df)

42141

In [4]:
len(df)

42141

In [5]:
df.lead_sponsor_type.value_counts()

academique    22412
industriel    19725
Name: lead_sponsor_type, dtype: int64

In [6]:
#df[df.lead_sponsor.apply(lambda x:'APHP' in str(x))].lead_sponsor.value_counts()

In [7]:
mapping = pd.read_csv('https://raw.githubusercontent.com/dataesr/bso-clinical-trials/main/bsoclinicaltrials/server/main/bso-lead-sponsors-mapping.csv')

In [8]:
list_rors = [r.split('/')[-1] for r in mapping.ror.unique() if (r==r) and len(r.split('/')[-1])==9]
len(list_rors)

102

In [9]:
df_completed_intervention_all = df[(df.status_simplified=='Completed') & (df.study_type=='Interventional')]

df_completed_intervention = df[(df.status_simplified=='Completed') & (df.study_type=='Interventional') & (df.lead_sponsor_type=='academique')]

mycols = ['lead_sponsor', 'study_completion_year', 'NCTId', 'eudraCT', 'study_type', 
                                 'status_simplified', 'acronym', 'title',
    'has_results_or_publications', 'has_results_or_publications_within_1y', 'has_results_or_publications_within_3y']

df2 = df_completed_intervention[mycols]

In [10]:
excluded_rors = [
    '02n6c9837', #sanofi
    '02xnj2427'
]

In [11]:
df_intervention = df[ (df.study_type=='Interventional') & (df.lead_sponsor_type=='academique')]

dd_perim = pd.merge(df_intervention[mycols], mapping, left_on='lead_sponsor', right_on='sponsor', indicator=True, how='inner')
dd_perim_ror = dd_perim[dd_perim.ror.apply(lambda x:isinstance(x, str) and 'ror.org/' in x)]
dd_perim_ror['ror_simple'] = dd_perim_ror.ror.apply(lambda x:x.split('/')[-1])
dd_perim_ror.ror_simple.value_counts().head(65)
df_perimetre = pd.DataFrame(dd_perim_ror.ror_simple.value_counts().head(64)).reset_index() # 10 essais 
df_perimetre.columns = ['ror', 'nb']
df_perimetre = df_perimetre[df_perimetre.ror.apply(lambda x:x not in excluded_rors)]
df_name = dd_perim_ror[['ror_simple', 'sponsor_normalized']].drop_duplicates()
df_perimetre2 = pd.merge(df_perimetre, df_name, left_on='ror', right_on='ror_simple', how='left')
df_perimetre2[['ror', 'nb', 'sponsor_normalized']].to_csv('../publipostage/perimetre.csv', index=False)

In [12]:
rors_to_compute = df_perimetre.ror.to_list()

In [13]:
dd = pd.merge(df2, mapping, left_on='lead_sponsor', right_on='sponsor', indicator=True, how='inner')

In [14]:
dd._merge.value_counts()

both          6005
left_only        0
right_only       0
Name: _merge, dtype: int64

In [15]:
dd_ror = dd[dd.ror.apply(lambda x:isinstance(x, str) and 'ror.org/' in x)]

In [16]:
dd_ror['ror_simple'] = dd_ror.ror.apply(lambda x:x.split('/')[-1])

In [17]:
# APHP 00pg5jh14 
# IPC 04s3t1g37

In [18]:
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
SIZE = 22
SMALL_SIZE=18
plt.rc('font', size=SIZE)          # controls default text sizes
plt.rc('axes', titlesize=SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SIZE)    # legend fontsize
plt.rc('figure', titlesize=SIZE)  # fontsize of the figure title  

In [19]:
cle_false = 'SANS communication\nidentifiée'
cle_true = 'AVEC communication\nidentifiée'

LAST_YEAR = 2022

def analyze_global(df_etab):
    ans = {'global': {}, '10Y': {}}
    
    d_g = pd.DataFrame(df_etab.has_results_or_publications.value_counts(dropna=False))
    d_g_10 = pd.DataFrame(df_etab[(df_etab.study_completion_year > LAST_YEAR - 10) & (df_etab.study_completion_year <= LAST_YEAR)].has_results_or_publications.value_counts(dropna=False))
    
    try:
        ans['global'][cle_true] = d_g[d_g.index==True].has_results_or_publications.values[0]
    except:
        ans['global'][cle_true] = 0
    try:
        ans['global'][cle_false] = d_g[d_g.index==False].has_results_or_publications.values[0]
    except:
        ans['global'][cle_false] = 0
    #ans['global']['percentage'] = round(100 * ans['global'][cle_true] / ans['global'][cle_false], 0)
    assert(ans['global'][cle_false] + ans['global'][cle_true]) == len(df_etab)
    

    try:
        ans['10Y'][cle_true] = d_g_10[d_g_10.index==True].has_results_or_publications.values[0]
    except:
        ans['10Y'][cle_true] = 0
    try:
        ans['10Y'][cle_false] = d_g_10[d_g_10.index==False].has_results_or_publications.values[0]
    except:
        ans['10Y'][cle_false] = 0
    #ans['10Y']['percentage'] = round(100 * ans['10Y'][cle_true] / ans['10Y'][cle_false], 0)
    
    return ans


In [20]:
def make_evolution(ror, df_tmp):
    
    df1Y = pd.DataFrame(df_tmp[(df_tmp.study_completion_year>=2014) & (df_tmp.study_completion_year<=LAST_YEAR)]\
    .groupby('study_completion_year')['has_results_or_publications_within_1y', ].mean()).reset_index()
    #df3Y = pd.DataFrame(df_tmp[(df_tmp.study_completion_year>=2014) & (df_tmp.study_completion_year<2021)]\
    #.groupby('study_completion_year')['has_results_or_publications_within_3y', ].mean()).reset_index()
    #df1_3Y = pd.merge(df1Y, df3Y, on='study_completion_year', how='left')
    df1_3Y = df1Y
    years = df1_3Y.study_completion_year.tolist()
    rate1Y = df1_3Y.has_results_or_publications_within_1y.tolist()
    #rate3Y = df1_3Y.has_results_or_publications_within_3y.tolist()
    #rate3Y = [r for r in rate3Y if r==r]
    
    fig, ax = plt.subplots(figsize=(9,9), dpi=100)

    width = 0.75  # the width of the bars
    multiplier = 0

    #offset1 = width * multiplier
    #rects1 = ax.bar([y-0.1 for y in years], [100 * r for r in rate1Y], width, label='1 an')
    #ax.bar_label(rects1, labels = [f'{round(100*r)} %' for r in rate1Y], padding=3, fontsize=SMALL_SIZE)

    #multiplier = 1
    #offset2 = width * multiplier
    #rects2 = ax.bar([y+0.1 for y in years[0:len(rate3Y)]], [100 * r for r in rate3Y], width, label='3 ans')
    #ax.bar_label(rects2, labels = [f'{round(100*r)} %' for r in rate3Y], padding=3, fontsize=SMALL_SIZE)

    offset1 = width * multiplier
    rects1 = ax.bar([y for y in years], [100 * r for r in rate1Y], width, label='1 an', color=['#DB87C9'])
    ax.bar_label(rects1, labels = [f'{round(100*r)} %' for r in rate1Y], 
                 padding=3, 
                 fontsize=SMALL_SIZE)

    

    # Add some text for labels, title and custom x-axis tick labels, etc.
    #ax.set_ylabel('Length (mm)')
    ax.yaxis.set_major_formatter(mtick.PercentFormatter())
    #ax.set_title('Penguin attributes by species')
    ax.set_xticks(years)
    ax.set_xticklabels([str(y).replace('.0', '') for y in years], rotation='vertical', fontsize=SMALL_SIZE)
    ax.legend(loc='upper left')
    ax.set_xlabel("Année de clôture de l'essai clinique")

    try:
        ax.set_ylim(0, min(100, max(rate3Y)*100+15))
    except:
        ax.set_ylim(0, 100)

    plt.savefig(f"./../publipostage/{ror}/evolution_temporelle_essais_cliniques_{ror}.png", bbox_inches="tight", dpi=300)

In [21]:
def get_perc(stats):
    true_global = int(round(100*stats['global'][cle_true]/(stats['global'][cle_false]+stats['global'][cle_true]), 0))
    false_global = int(round(100*stats['global'][cle_false]/(stats['global'][cle_false]+stats['global'][cle_true]), 0))
    true_10Y = int(round(100*stats['10Y'][cle_true]/(stats['10Y'][cle_false]+stats['10Y'][cle_true]), 0))
    false_10Y = int(round(100*stats['10Y'][cle_false]/(stats['10Y'][cle_false]+stats['10Y'][cle_true]), 0))
    return f'{true_global} %', f'{true_10Y} %'

In [22]:
print('france')
stat_france = analyze_global(df_completed_intervention_all)
print(get_perc(stat_france))

print('academique')
stat_academique = analyze_global(df_completed_intervention)
print(get_perc(stat_academique))

france
('54 %', '52 %')
academique
('29 %', '28 %')


In [23]:
df[['title', 'acronym']]

,title,acronym
0,Long-term Effects of Aldara® 5% Cream and Sola...,LEIDA
1,Testing for Increased Lipoproteins (a) [LP(a)]...,LiLY
2,"Interventional, Randomized, Double-blind, Plac...",NaN
3,"A Prospective, Multicentre Study Evaluating th...",NaN
4,"An Open-label, Long-term Safety and Efficacy S...",D2306
...,...,...
42728,"A Randomized, Active Controlled, Parallel Grou...",RA-BRIDGE
42729,"A Phase 3, Randomized, Double-blind, Placebo-c...",NaN
42927,"A Phase 1b/2, Open-Label, Safety and Efficacy ...",EPCORE™ CLL-1
43068,"A Randomized, Phase 2, Double-blind Study to E...",213403


In [24]:
def get_score(r):
    if r.has_results_or_publications_within_1y is False and r.has_results_or_publications_within_3y is False:
        return "\U0001f7E5"
    if r.has_results_or_publications_within_1y is False and r.has_results_or_publications_within_3y is True:
        return "\U0001f7E7"
    if r.has_results_or_publications_within_1y is True and r.has_results_or_publications_within_3y is True:
        return "\U0001f7E9"

In [25]:
def clean_year(y):
    try:
        return str(int(y))
    except:
        return None

In [26]:
import os

def make_data(ror):
    os.system(f"mkdir -p ./../publipostage/{ror}")
    
    
    df_tmp = dd_ror[dd_ror.ror_simple == ror].sort_values(by='study_completion_year')
    dx = pd.DataFrame(df_tmp.lead_sponsor.value_counts()).reset_index()
    dx.columns = ['variant', 'nb_studies']
    dx[['variant']].to_csv(f'./../publipostage/{ror}/liste_variantes_noms_{ror}.csv', index=False, header=False, sep='£')
    df_tmp['S'] = df_tmp.apply(lambda row: get_score(row), axis=1) 
    cols = ['S', 'NCTId', 'eudraCT', 'study_completion_year', 'title', 'acronym', 'has_results_or_publications_within_1y',
           'has_results_or_publications_within_3y']
    stats = analyze_global(df_tmp)
    #print(stats)
    
    global_stat = {'ror': ror}
    try:
        global_stat['sponsor_name'] = df_tmp.sponsor_normalized.values[0]
    except:
        print('skip '+ror)
        return
    global_stat['sponsor_article'] = 'du'
    if global_stat['sponsor_name'][0] in ['A', 'E', 'I', 'O', 'U', 'Y']:
        global_stat['sponsor_article'] = "de l'"
    if global_stat['sponsor_name'].split(' ')[0].lower() in ['hospices', 'hopitaux', 'hôpitaux']:
        global_stat['sponsor_article'] = 'des'
    if global_stat['sponsor_name'].split(' ')[0].lower() in ['hopital', 'hôpital']:
        global_stat['sponsor_article'] = "de l'"
    if global_stat['sponsor_name'].split(' ')[0].lower() in ['clinique']:
        global_stat['sponsor_article'] = "de la"
    a, b = get_perc(stats)
    global_stat['global_indicator'] = a
    global_stat['10Y_indicator'] = b
    pd.DataFrame([global_stat]).to_csv(f'./../publipostage/{ror}/indicators_{ror}.csv', index=False)
    
    print(global_stat['sponsor_article'], global_stat['sponsor_name'])
    
    make_pie(ror, stats, 'global')
    make_pie(ror, stats, '10Y')
    make_evolution(ror, df_tmp)
    plt.close('all')
    
    df_tmp['study_completion_year'] = df_tmp['study_completion_year'].apply(lambda x:clean_year(x))
    df_liste_essais = df_tmp[cols]
    df_liste_essais.columns = ['statut', 'NCTId', 'eudraCT', 'completion_year', 
                               'clinical_trial_title', 'acronym', 
                               'results_1y', 'results_3y']
    df_liste_essais.to_csv(f'./../publipostage/{ror}/liste_essais_cliniques_identifies_{ror}.csv', index=False)
    

def make_pie(ror, stats, pie_type):
    
    fig, ax = plt.subplots(figsize=(9,9), dpi=100)
    values = list(stats[pie_type].values())
    labels_raw=list(stats[pie_type].keys())
    total = values[0] + values[1]
    labels = [f'{values[0]} essais cliniques sur {total}\nSANS communication identifiée',
             f'{values[1]} essais cliniques sur {total}\nAVEC communication identifiée']

    colors=['#FFF5BA', '#5A35BE']

    _, _, autotexts  = ax.pie(values, startangle=90, autopct='%1.0f%%', 
                 explode=(0.025, 0.025),
                 colors=colors)

    autotexts[1].set_color('white')

    ax.legend(labels=labels, bbox_to_anchor=(0.15,0.5), 
              loc="center right", fontsize=20, 
               bbox_transform=plt.gcf().transFigure)

    
    plt.savefig(f"./../publipostage/{ror}/repartition_essais_cliniques_{pie_type}_{ror}.png", bbox_inches="tight", dpi=300)


In [27]:
len(rors_to_compute)

62

In [ ]:
for ix, current_ror in enumerate(rors_to_compute):
    
    if current_ror in excluded_rors:
        continue
    print(ix, current_ror, end=' ...')
    
    make_data(current_ror)

    #if ix > 10:
    #    break
    
#make_data('04s3t1g37')

0 00pg5jh14 ...de l' Assistance Publique – Hôpitaux de Paris
1 01502ca60 ...des Hospices Civils de Lyon
2 00mthsf17 ...du Centre Hospitalier Universitaire de Montpellier
3 017h5q109 ...du Centre hospitalier universitaire de Toulouse
4 02tcf7a68 ...du Centre hospitalier universitaire de Clermont-Ferrand
5 002cp4060 ...de l' Assistance publique - Hôpitaux de Marseille
6 01hq89f96 ...du Centre hospitalier universitaire de Bordeaux
7 041rhpw39 ...du Centre hospitalier universitaire Grenoble-Alpes
8 05c1qsg97 ...du Centre hospitalier universitaire de Nantes
9 05qsjq305 ...du Centre hospitalier universitaire de Nice
10 04bckew43 ...des Hôpitaux universitaires de Strasbourg
11 04pn6vp43 ...du Centre hospitalier universitaire de Saint-Étienne
12 010567a58 ...du Centre hospitalier universitaire d'Amiens - Picardie
13 02ppyfa04 ...du Centre hospitalier régional universitaire de Lille
14 027arzy69 ...du Centre hospitalier universitaire de Caen
15 05qec5a53 ...du Centre hospitalier universitaire d